In [8]:
import tensorflow as tf
import zipfile
import lxml.etree
import re


## Data Preparation

In [12]:
def parse(doc):
    return [(f.xpath("./content/text()")[0], f.xpath("./head/keywords/text()")[0]) for f in doc.xpath("//file")]

def label(keywords):
    kwset = set([kw.strip().lower() for kw in keywords.split(",")])
    return 4 * ("technology" in kwset) + 2 * ("entertainment" in kwset) + 1 * ("design" in kwset)

def normalise(text):
    input_text_noparens = re.sub(r'\([^)]*\)', '', text)
    no_speakers = []
    for line in input_text_noparens.split('\n'):
        m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
        no_speakers.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

    return re.sub(r"[^a-z0-9]+", " ", ' '.join(no_speakers).lower()).split()

with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))

all_data = [(normalise(content), label(keywords)) for (content, keywords) in parse(doc)]
del doc


Split into training/validation/test sets:

In [13]:
training = all_data[:1585]
validation = all_data[1585:1585+250]
test = all_data[1585+250:]

## Model

**x** = embedding(*text*)<br>
**h** = tanh(**Wx** + **b**)<br>
**u** = **Vh** + **c**<br>
**p** = softmax(**u**)<br>
if testing:<br>
&nbsp;&nbsp;&nbsp;&nbsp;prediction = arg max<sub>y’</sub> p<sub>y’</sub><br>
else: # training, with y as the given gold label<br>
&nbsp;&nbsp;&nbsp;&nbsp;loss = -log(p<sub>y</sub>)  # cross entropy criterion

In [ ]:
vocabulary_size


embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

x = tf.placeholder(tf.float32, [None, ])
